In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np

In [7]:
# Load CIFAR-10 dataset directly from Keras
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()


In [8]:
# Normalize the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [9]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [10]:
# Load VGG16 without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


In [11]:
# Freeze parameters (weights) in model’s lower convolutional layers
for layer in base_model.layers:
    layer.trainable = False

In [12]:
# Add custom classifier with several layers of trainable parameters to the model
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
predictions = Dense(10, activation='softmax')(x)

In [13]:
# Create the model
model = Model(inputs=base_model.input, outputs=predictions)


In [14]:
# Compile the model
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10


111/782 [===>..........................] - ETA: 5:10 - loss: 1.8781 - accuracy: 0.3359

In [ ]:
# Fine-tune hyperparameters and unfreeze more layers as needed
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


In [ ]:
# Freeze all layers first
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Unfreeze last 4 layers of base model
for layer in base_model.layers[len(base_model.layers) - 4:]:
    layer.trainable = True

In [ ]:
# Fine-tuning hyperparameters
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
predictions = Dense(10, activation='softmax')(x)

In [ ]:
# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# Compile the fine-tuned model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the fine-tuned model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))


In [ ]:
# Evaluate the model
import matplotlib.pyplot as plt

In [ ]:
predicted_value = model.predict(x_test)
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [ ]:
n = 890
plt.imshow(x_test[n])
print("Predicted: ", labels[np.argmax(predicted_value[n])])
print("Actual: ", labels[np.argmax(y_test[n])])